In [1]:
import pandas as pd
import geopandas as gpd
import shapely
from tqdm.auto import tqdm

In [2]:
def geocode(data, api_key):
    """

    Args:
        data (_type_): _description_
        api_key (_type_): _description_

    Returns:
        _type_: _description_
    """
    from datetime import datetime
    import pandas as pd
    import requests

    print(f"Started geocoding {data['parsed_add'].nunique()} cities")
        # Required fields in data df: 'l_id'; 'parsed_add'
    
    now = datetime.now()
    log_time = str(now.strftime("%m_%d_%Y"))
    n=0
    retries =[]
    for i in range(len(data)):
        # Set the address that you want to geocode
        address = data['parsed_add'][i]
            
            # Build the API request URL with the address and boundary
        url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"

            # Send the request and get the response
        response = requests.get(url).json()

        data.loc[i, 'status'] = response["status"]
        try:
                # Get the latitude and longitude from the response
            location = response['results'][0]['geometry']['location']
            latitude = location['lat']
            longitude = location['lng']
            data.loc[i, 'lat'] = latitude
            data.loc[i, 'long'] = longitude 
        except Exception as e:
            retries.append(data['parsed_add'][i])

        n += 1
        if n % 100 == 0:  # Check if current index is a multiple of 500
            print(f"Processed {n} cities")
            #print(data.loc[i])
            data.to_csv(f'data/result_geocode_{log_time}.csv',index=False)
        
    # Iterate through the list and write each element to the file
    with open(f'data/failed_ids_{log_time}.txt', 'w') as file:
        for item in retries:
            file.write("%s\n" % item)
    data.to_csv(f'data/result_geocode_{log_time}.csv',index=False)
    return data, retries

In [3]:
'''d = pd.DataFrame({'parsed_add': [x + ' California' for x in lookups[lookups['field_long'] == 'City']['lookup_long']],
                  'lat': [0] * len(lookups[lookups['field_long'] == 'City']),
                  'long': [0] * len(lookups[lookups['field_long'] == 'City']),
                  'status': [None] * len(lookups[lookups['field_long'] == 'City'])})
import json
with open('data/api_keys/keys.json', 'r') as f:
    k = json.load(f)
#d, retries = geocode(d, k['google_maps'])
'''

"d = pd.DataFrame({'parsed_add': [x + ' California' for x in lookups[lookups['field_long'] == 'City']['lookup_long']],\n                  'lat': [0] * len(lookups[lookups['field_long'] == 'City']),\n                  'long': [0] * len(lookups[lookups['field_long'] == 'City']),\n                  'status': [None] * len(lookups[lookups['field_long'] == 'City'])})\nimport json\nwith open('data/api_keys/keys.json', 'r') as f:\n    k = json.load(f)\n#d, retries = geocode(d, k['google_maps'])\n"

In [4]:
def get_radius(
    listings: gpd.GeoDataFrame, centroid: shapely.Point, thresh=0.75, radius_incr=0.005
) -> float:
    if len(listings) == 0:
        return 0
    if centroid == shapely.Point(0, 0) or pd.isna(centroid):
        centroid = shapely.MultiPoint(listings["geometry"].values).centroid
    
    radius = 0 + radius_incr
    check = gpd.GeoDataFrame(
        {"check": [0], "geometry": [centroid.buffer(radius)]}, crs="EPSG:4326"
    )
    while len(listings.sjoin(check, predicate="intersects")) / len(listings) < thresh:
        radius += radius_incr
        check = gpd.GeoDataFrame(
            {"check": [0], "geometry": [centroid.buffer(radius)]}, crs="EPSG:4326"
        )
    # print(len(listings.sjoin(check, predicate='intersects'))/len(listings))
    return radius

In [5]:
def get_centroids(listings):
    return shapely.MultiPoint(listings["geometry"].values).centroid

In [6]:
df = pd.concat([pd.read_csv('data/cincy_raw_data.csv'), pd.read_csv('data/cincy_raw_data_2.csv')])
df = df[df['latitude']!=0]
len(df)

/var/folders/6l/yzmcq93j41zf3wrth_tyl6g80000gn/T/ipykernel_45668/570044259.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv('data/cincy_raw_data.csv'), pd.read_csv('data/cincy_raw_data_2.csv')])


278101

In [7]:
df.head()

,source_id,city,postal_code,mls_area_major,county_or_parish,subdivision_name,township,latitude,longitude
17,485142,Hamilton Twp,45152,Warren-E09,Warren,Villages of Classic Way,NaN,39.351799,-84.131096
21,486571,Colerain Twp,45231,Hamilton-W10,Hamilton,NaN,Colerain,39.251793,-84.568352
24,486947,West Chester,45069,Butler-E12,Butler,NaN,NaN,39.320988,-84.399513
30,1234906,Oxford,45056,Butler-W18,Butler,NaN,Oxford,39.494304,-84.748276
34,485130,Hamilton Twp,45152,Warren-E09,Warren,Villages of Classicway,NaN,39.351799,-84.131096


In [8]:
df["geometry"] = [
        shapely.Point(long, lat)
        for lat, long in tqdm(zip(df["longitude"], df["latitude"]), total=len(df))
    ]
df = gpd.GeoDataFrame(df, crs="EPSG:4326")

  0%|          | 0/278101 [00:00<?, ?it/s]

In [9]:
df.columns

Index(['source_id', 'city', 'postal_code', 'mls_area_major',
       'county_or_parish', 'subdivision_name', 'township', 'latitude',
       'longitude', 'geometry'],
      dtype='object')

In [10]:
fields = ['city', 'postal_code', 'mls_area_major', 'county_or_parish', 'subdivision_name', 'township']

In [11]:
for field in fields:
    print(field)
    tmp = pd.DataFrame({f'{field}': df[f'{field}'].unique()})
    tmp.dropna(inplace=True)
    centroids = [
        shapely.MultiPoint(df[df[f'{field}'] == area]["geometry"].values).centroid
        for area in tqdm(
            tmp[f'{field}'], total=len(tmp[f'{field}'])
        )
    ]
    tmp["centroid"] = centroids

    radii = [
        get_radius(df[df[f'{field}'] == l], c)
        for l, c in tqdm(
            zip(tmp[f'{field}'], tmp["centroid"]),
            total=len(tmp),
        )
    ]
    tmp["radius"] = radii
    tmp.to_csv(f'data/cincy_{field}.csv')

city


  0%|          | 0/452 [00:00<?, ?it/s]

  0%|          | 0/452 [00:00<?, ?it/s]

postal_code


  0%|          | 0/794 [00:00<?, ?it/s]

...

  0%|          | 0/17702 [00:00<?, ?it/s]

  0%|          | 0/17702 [00:00<?, ?it/s]

township


  0%|          | 0/1667 [00:00<?, ?it/s]

  0%|          | 0/1667 [00:00<?, ?it/s]